<a href="https://colab.research.google.com/github/Toto17515/KKU_Project/blob/Data/Indicator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Algorithm

##Library

In [1]:
#Basic
import math
import decimal
import statsmodels
import pandas as pd
import numpy as np
import matplotlib as mpl
import statsmodels.api as sm
from itertools import combinations
from math import isnan, nan
from datetime import datetime 
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


##Sub Function

In [2]:
#EMA
def EMA(Close, A):
  Close = Close.tolist()
  if len(Close) > A-1:
    EMA = []; ws = 0; wc = 0
    for a in range(A-1):
      EMA.append(float('NaN'))
    alpha = 2/(A+1)
    for b in range(A):
      ws = ws + Close[b]*(1-alpha)**b
      wc = wc + (1-alpha)**b
    EMA.append(ws/wc)
    for c in range(len(Close)-A):
      EMA.append(EMA[c+A-1]+alpha*(Close[c+A]-EMA[c+A-1]))
    if Close[len(Close)-1] > EMA[-1]: return 'Up'
    elif Close[len(Close)-1] < EMA[-1]: return 'Down'
    else: return 'Sideway'

In [3]:
#PSAR
def PSAR(High, Low):
  High = High.tolist()
  Low = Low.tolist()
  sar = [float('NaN'),float('NaN')]
  direction = 'None'; h = 0; k = 0; d = 2; AF = 0.02
  if len(High) == len(Low) and len(High)> 2:
    #Begin
    for i in range(len(High)-1):
      if (High[i+1] > High[i] and Low[i+1] < Low[i]) or (High[i+1] < High[i] and Low[i+1] > Low[i]):
        direction = 'Sideway'
        sar.append(float('NaN'))
        d += 1
      elif  High[i+1] > High[i]:
        direction = 'Up'
        h = max(High[:i+d+1])
        sar.append(min(Low[:i+1])); break
      else:
        direction = 'Down'
        l = min(Low[:i+d+1])
        sar.append(max(High[:i+1])); break
    #PSAR
    for j in range(len(High)-d):
      if direction == 'Up':
        psar = sar[j+d]+AF*(h-sar[j+d])
        if Low[j+d] > psar: 
          sar.append(psar)
          psar_trend = 'Up'
          direction = 'Up'
          if j < len(High)-d-1:
            if h < High[j+d+1]: 
              h = High[j+d+1]
              if AF < 0.20: AF += 0.02
        else: 
          sar.append(h)
          psar_trend = 'Down'
          direction = 'Down'
          AF = 0.02
          if j < len(High)-d-1: l = Low[j+d+1]
      else:
        psar = sar[j+d]-AF*(sar[j+d]-l)
        if High[j+d] < psar: 
          sar.append(psar)
          psar_trend = 'Down'
          direction = 'Down'
          if j < len(Low)-d-1:
            if l > Low[j+d+1]: 
              l = Low[j+d+1]
              if AF < 0.20: AF += 0.02
        else: 
          sar.append(l)
          psar_trend ='Up'
          direction = 'Up'
          AF = 0.02   
          if j < len(Low)-d-1: h = High[j+d+1]     
    return psar_trend

In [4]:
#RSI
def RSI(Close, B):
  Close = Close.tolist()
  Diff = []; Up = []; Down = []; 
  UpAVG = []; DownAVG = []; RSI = []; 
  for a in range(B):
    RSI.append(float('NaN'))
  for i in range(len(Close)):
    if i > 0: Diff.append(Close[i]-Close[i-1])
  for j in range(len(Diff)):
    if Diff[j] > 0:
      Up.append(Diff[j])
      Down.append(0)
    elif Diff[j] < 0:
      Up.append(0)
      Down.append(-Diff[j])
    else:
      Up.append(0)
      Down.append(0)
  UpAVG.append(sum(Up[:B])/len(Up[:B]))
  DownAVG.append(sum(Down[:B])/len(Down[:B]))
  for k in range(len(Diff)-B):
    UpAVG.append((UpAVG[k]*(B-1)+Up[k+B])/B)
    DownAVG.append((DownAVG[k]*(B-1)+Down[k+B])/B)
  for l in range(len(UpAVG)):
    if DownAVG[l] != 0:
      RSI.append(100-100/(1+(UpAVG[l]/DownAVG[l])))
    else: RSI.append(100)
  if RSI[-1] > 50: return 'Up'
  elif RSI[-1] < 50: return 'Down'
  else: return 'Sideway'

In [5]:
#WPR
def WPR(High, Low, Close, C):
  High = High.tolist()
  Low = Low.tolist()
  Close = Close.tolist()
  Highest = max(High[-C:]) 
  Lowest = min(Low[-C:])
  WPR= (Close[len(Close)-1]-Highest)/(Highest-Lowest)*100
  if WPR > -50: return 'Up'
  elif WPR < -50: return 'Down'
  else: return 'Sideway' 

##Main Function

In [6]:
#Indicator
def Indicator_Algorithm(FOREX_High, FOREX_Low, FOREX_Close, A, B, C):
  Y1 = FOREX_High; Y2 = FOREX_Low; Y3 = FOREX_Close
  Indicator = 'None'
  if len(Y1) == len(Y2) == len(Y3):  
    if EMA(Y3,A) == PSAR(Y1,Y2) == RSI(Y3,B) == WPR(Y1,Y2,Y3,C): Indicator = EMA(Y3,A)
    else: Indicator = 'Sideway'
    return Indicator
  else: return 'The data size must be the same.'